In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Model Evaluation and Validation
# Purpose: Evaluate and validate machine learning models for predicting rainfall_sum and extreme_rainfall in Eastern Nepal.


# Set output directory
OUTPUT_DIR = '../Data/Preprocessed'
MODEL_DIR = '../Outputs'

# Step 1: Load feature-engineered data
data_path = os.path.join(OUTPUT_DIR, 'feature_engineered_data.csv')
try:
    data = pd.read_csv(data_path, low_memory=False)
except FileNotFoundError:
    print(f"Error: The file {data_path} does not exist. Please ensure the file is in the correct directory.")
    exit()

# Step 2: Split data into training and validation sets
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size=0.2, shuffle=False)

# Step 3: Prepare features and targets
# Define columns to exclude (non-numeric or irrelevant)
exclude_columns = [
    'rainfall_sum', 'extreme_rainfall', 'station_name_x', 'district_x', 'station_name_y',
    'basin_office', 'types_of_station', 'district_y', 'unnamed:_9_y', 'date'
]

# Select only numeric columns and exclude target columns
numeric_columns = data.select_dtypes(include=[np.number]).columns
feature_columns = [col for col in numeric_columns if col not in exclude_columns]

# Prepare training and validation data
try:
    X_train_reg = train_data[feature_columns]
    y_train_reg = train_data['rainfall_sum']
    X_val_reg = val_data[feature_columns]
    y_val_reg = val_data['rainfall_sum']

    X_train_clf = train_data[feature_columns]
    y_train_clf = train_data['extreme_rainfall']
    X_val_clf = val_data[feature_columns]
    y_val_clf = val_data['extreme_rainfall']
except KeyError as e:
    print(f"Error: Column not found - {e}. Please check the column names in the dataset.")
    exit()

# Debug: Print data types to ensure all are numeric
print("Training data types (Regression):\n", X_train_reg.dtypes)
print("Validation data types (Regression):\n", X_val_reg.dtypes)
print("Training data types (Classification):\n", X_train_clf.dtypes)
print("Validation data types (Classification):\n", X_val_clf.dtypes)

# Check for non-numeric columns
non_numeric_reg = X_val_reg.select_dtypes(exclude=[np.number]).columns
if non_numeric_reg.size > 0:
    print(f"Error: Non-numeric columns in regression validation data: {non_numeric_reg}")
    exit()

non_numeric_clf = X_val_clf.select_dtypes(exclude=[np.number]).columns
if non_numeric_clf.size > 0:
    print(f"Error: Non-numeric columns in classification validation data: {non_numeric_clf}")
    exit()

# Check for missing values
if X_val_reg.isnull().sum().sum() > 0:
    print("Missing values in regression validation data:", X_val_reg.isnull().sum())
    X_val_reg = X_val_reg.fillna(X_val_reg.mean())

if X_val_clf.isnull().sum().sum() > 0:
    print("Missing values in classification validation data:", X_val_clf.isnull().sum())
    X_val_clf = X_val_clf.fillna(X_val_clf.mean())

# Step 4: Load trained models
best_reg_model_path = os.path.join(MODEL_DIR, 'best_random_forest_regressor_model.pkl')
best_clf_model_path = os.path.join(MODEL_DIR, 'best_random_forest_classifier_model.pkl')

try:
    with open(best_reg_model_path, 'rb') as f:
        best_reg_model = pickle.load(f)
    with open(best_clf_model_path, 'rb') as f:
        best_clf_model = pickle.load(f)
except FileNotFoundError as e:
    print(f"Error: Model file not found - {e}")
    exit()
except Exception as e:
    print(f"Error loading models: {e}. Ensure models are compatible with the current scikit-learn version.")
    exit()

# Verify feature consistency with trained model
if hasattr(best_reg_model, 'feature_names_in_'):
    expected_features = list(best_reg_model.feature_names_in_)
    if set(X_val_reg.columns) != set(expected_features):
        print(f"Error: Feature mismatch. Expected: {expected_features}, Got: {X_val_reg.columns}")
        exit()
    X_val_reg = X_val_reg[expected_features]  # Ensure correct order
    X_val_clf = X_val_clf[expected_features]

# Step 5: Evaluate models on validation set
# Regression evaluation
try:
    y_pred_reg = best_reg_model.predict(X_val_reg)
    mae = mean_absolute_error(y_val_reg, y_pred_reg)
    rmse = np.sqrt(mean_squared_error(y_val_reg, y_pred_reg))
    r2 = r2_score(y_val_reg, y_pred_reg)
    print("Regression Model - Validation Set:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
except Exception as e:
    print(f"Error in regression prediction: {e}")
    exit()

# Classification evaluation
try:
    y_pred_clf = best_clf_model.predict(X_val_clf)
    accuracy = accuracy_score(y_val_clf, y_pred_clf)
    precision = precision_score(y_val_clf, y_pred_clf, zero_division=0)
    recall = recall_score(y_val_clf, y_pred_clf, zero_division=0)
    f1 = f1_score(y_val_clf, y_pred_clf, zero_division=0)
    roc_auc = roc_auc_score(y_val_clf, best_clf_model.predict_proba(X_val_clf)[:, 1])
    print("Classification Model - Validation Set:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
except Exception as e:
    print(f"Error in classification prediction: {e}")
    exit()

# Step 6: Cross-validation
tscv = TimeSeriesSplit(n_splits=5)
try:
    cv_r2 = cross_val_score(best_reg_model, X_train_reg, y_train_reg, cv=tscv, scoring='r2')
    print("Cross-Validation R2 Scores (Regression):", cv_r2)
    print(f"Mean R2: {cv_r2.mean():.4f}")
    print(f"Std R2: {cv_r2.std():.4f}")
except Exception as e:
    print(f"Error in regression cross-validation: {e}")

try:
    cv_f1 = cross_val_score(best_clf_model, X_train_clf, y_train_clf, cv=tscv, scoring='f1')
    print("Cross-Validation F1 Scores (Classification):", cv_f1)
    print(f"Mean F1: {cv_f1.mean():.4f}")
    print(f"Std F1: {cv_f1.std():.4f}")
except Exception as e:
    print(f"Error in classification cross-validation: {e}")

# Step 7: Baseline models
# Regression baseline (predict mean)
mean_pred = np.mean(y_train_reg)
baseline_mae = mean_absolute_error(y_val_reg, [mean_pred] * len(y_val_reg))
baseline_rmse = np.sqrt(mean_squared_error(y_val_reg, [mean_pred] * len(y_val_reg)))
baseline_r2 = r2_score(y_val_reg, [mean_pred] * len(y_val_reg))
print("Baseline Regression (Mean Prediction):")
print(f"MAE: {baseline_mae:.4f}")
print(f"RMSE: {baseline_rmse:.4f}")
print(f"R2: {baseline_r2:.4f}")

# Classification baseline (majority class)
majority_class = y_train_clf.mode()[0]
baseline_accuracy = accuracy_score(y_val_clf, [majority_class] * len(y_val_clf))
print("Baseline Classification (Majority Class):")
print(f"Accuracy: {baseline_accuracy:.4f}")

# Step 8: Regional evaluation
val_data = val_data.copy()  # Avoid SettingWithCopyWarning
val_data['pred_rainfall'] = y_pred_reg
val_data['pred_extreme'] = y_pred_clf

if 'station_id' in val_data.columns:
    try:
        regional_performance_reg = val_data.groupby('station_id').apply(lambda x: pd.Series({
            'MAE': mean_absolute_error(x['rainfall_sum'], x['pred_rainfall']),
            'RMSE': np.sqrt(mean_squared_error(x['rainfall_sum'], x['pred_rainfall'])),
            'R2': r2_score(x['rainfall_sum'], x['pred_rainfall'])
        }))
        print("Regional Performance - Regression:")
        print(regional_performance_reg)

        regional_performance_clf = val_data.groupby('station_id').apply(lambda x: pd.Series({
            'Accuracy': accuracy_score(x['extreme_rainfall'], x['pred_extreme']),
            'Precision': precision_score(x['extreme_rainfall'], x['pred_extreme'], zero_division=0),
            'Recall': recall_score(x['extreme_rainfall'], x['pred_extreme'], zero_division=0),
            'F1': f1_score(x['extreme_rainfall'], x['pred_extreme'], zero_division=0)
        }))
        print("Regional Performance - Classification:")
        print(regional_performance_clf)

        # Visualize regional R2 for regression
        plt.figure(figsize=(10, 6))
        sns.barplot(x=regional_performance_reg.index, y=regional_performance_reg['R2'])
        plt.title('R2 Score by Station (Regression)')
        plt.xlabel('Station ID')
        plt.ylabel('R2 Score')
        plt.savefig(os.path.join(OUTPUT_DIR, 'regional_r2_plot.png'))
        plt.close()
    except Exception as e:
        print(f"Error in regional evaluation: {e}")
else:
    print("Warning: 'station_id' column not found. Skipping regional evaluation.")

# Step 9: Time period evaluation (by year)
if 'date' in val_data.columns:
    try:
        val_data['year'] = pd.to_datetime(val_data['date'], errors='coerce').dt.year
        if val_data['year'].isnull().all():
            print("Warning: Unable to parse dates. Skipping yearly evaluation.")
        else:
            yearly_performance_reg = val_data.groupby('year').apply(lambda x: pd.Series({
                'MAE': mean_absolute_error(x['rainfall_sum'], x['pred_rainfall']),
                'RMSE': np.sqrt(mean_squared_error(x['rainfall_sum'], x['pred_rainfall'])),
                'R2': r2_score(x['rainfall_sum'], x['pred_rainfall'])
            }))
            print("Yearly Performance - Regression:")
            print(yearly_performance_reg)
    except Exception as e:
        print(f"Error in yearly evaluation: {e}")
else:
    print("Warning: 'date' column not found. Skipping yearly evaluation.")

# Step 10: Save evaluation results
results = [
    {'Model': 'Best Regression Model', 'Metric': 'MAE', 'Value': mae},
    {'Model': 'Best Regression Model', 'Metric': 'RMSE', 'Value': rmse},
    {'Model': 'Best Regression Model', 'Metric': 'R2', 'Value': r2},
    {'Model': 'Best Classification Model', 'Metric': 'Accuracy', 'Value': accuracy},
    {'Model': 'Best Classification Model', 'Metric': 'Precision', 'Value': precision},
    {'Model': 'Best Classification Model', 'Metric': 'Recall', 'Value': recall},
    {'Model': 'Best Classification Model', 'Metric': 'F1', 'Value': f1},
    {'Model': 'Best Classification Model', 'Metric': 'ROC-AUC', 'Value': roc_auc},
    {'Model': 'Baseline Regression', 'Metric': 'MAE', 'Value': baseline_mae},
    {'Model': 'Baseline Regression', 'Metric': 'RMSE', 'Value': baseline_rmse},
    {'Model': 'Baseline Regression', 'Metric': 'R2', 'Value': baseline_r2},
    {'Model': 'Baseline Classification', 'Metric': 'Accuracy', 'Value': baseline_accuracy}
]

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(OUTPUT_DIR, 'model_evaluation_results.csv'), index=False)

if 'station_id' in val_data.columns:
    regional_performance_reg.to_csv(os.path.join(OUTPUT_DIR, 'regional_performance_regression.csv'))
    regional_performance_clf.to_csv(os.path.join(OUTPUT_DIR, 'regional_performance_classification.csv'))
if 'date' in val_data.columns and not val_data['year'].isnull().all():
    yearly_performance_reg.to_csv(os.path.join(OUTPUT_DIR, 'yearly_performance_regression.csv'))

print("Evaluation complete. Results saved to CSV files.")

Training data types (Regression):
 gsid                        int64
station_id                  int64
year                        int64
month                       int64
days                        int64
unnamed:_8                float64
unnamed:_9_x              float64
unnamed:_10               float64
s_n_                      float64
lat(deg)                  float64
lon(deg)                  float64
ele(meter)                float64
yearly_rainfall           float64
monthly_rainfall          float64
prev_day_rainfall         float64
rolling_mean_7d           float64
day_of_year               float64
log_rainfall_sum          float64
log_monthly_rainfall      float64
log_prev_day_rainfall     float64
log_rolling_mean_7d       float64
station_name_x_encoded      int64
pca_component_1           float64
pca_component_2           float64
pca_component_3           float64
dtype: object
Validation data types (Regression):
 gsid                        int64
station_id                  in